I'll refer to Gnedin et al. (2014) as "G14" hereafter.

In [ ]:
import os
import math

# Third-party
from astropy.visualization import hist
from astropy.constants import G
import astropy.units as u
import matplotlib.pyplot as pl
import numpy as np
pl.style.use('apw-notebook')
%matplotlib inline
from scipy.integrate import odeint, quad
from scipy.stats import powerlaw, rv_continuous
from scipy.optimize import root, minimize
from scipy.special import gamma, gammainc
import h5py

# Custom
import gala.coordinates as gc
import gala.dynamics as gd
import gala.integrate as gi
import gala.potential as gp
from gala.units import galactic

import biff.scf as bscf

In [ ]:
seed = 42 # for random numbers

In [ ]:
m_tot = 5E10 # Msun
R_e = 4. # kpc
n_s = 2.2

Oleg:

apologies for the delay.  I was traveling again.  I used the following routines to calculate the spherical Sersic profile:

```
void EllModel( double z, double *Msz, double *nserz, double *rserz,
           double *an, double *bn ) {
  double ns;
  *Msz = Ms*pow(1.+z, -0.67);
  *nserz = ns = nser*pow(1.+z, -0.95);
  *rserz = rser*pow(1.+z, -1.27);
  *an = 2.*ns;
  *bn = 2.*ns-1./3.+0.0098765/ns+0.0018/sqr(ns);
  return;
}

EllModel(z, &Msz, &nserz, &rserz, &an, &bn);
arg = bn*pow(r/rserz, 1./nserz);
Mstar = Msz*gammp_(&an, &arg);
```

gammp_() is the incomplete gamma function.


Everything is assumed to start at `z=3` and he says they fix the mass?

In [ ]:
z = 3.
m_tot*(1+z)**-0.67, n_s*(1+z)**-0.95, R_e*(1+z)**-1.27

### Solve for Sersic profile

using Eq. 4 of http://mnras.oxfordjournals.org/content/362/1/197.full.pdf

In [ ]:
# class SersicDensity(object):
#     """
#     Parameters
#     ----------
#     r : numeric, array_like
#         Spherical radius.
#     m : numeric
#         Total mass.
#     Re : numeric
#         Effective radius
#     n : numeric
#         Power law index?
#     """
    
#     def _density(self, r, rho0, Re, ns, p, b_n):
#         return rho0 * (r/Re)**-p * np.exp(-b_n*(r/Re)**(1/ns))
    
#     def __init__(self, m, Re, ns):
        
#         self.m = m
#         self.Re = Re
#         self.ns = ns
        
#         # first solve for b_n
#         def bn_func(b_n, n):
#             return 2 * gamma(2*n) * gammainc(n, b_n) - gamma(2*n)
#         res = root(bn_func, x0=0.1, args=(ns,))
#         self.b_n = res.x[0]

#         # now get p
# #         if ns < 0.6 or ns > 10: 
# #             raise ValueError("Invalid n")
# #         self.p = 1. - 0.6097/ns + 0.05563/ns**2
#         self.p = 0.

#         inv_rho0,_ = quad(lambda r,*args: 4*np.pi*r**2*self._density(r,*args), 0., np.inf, 
#                           args=(1., Re, ns, self.p, self.b_n))
#         self.rho0 = m / inv_rho0
    
#     def __call__(self, r):
#         return self._density(r, self.rho0, self.Re, self.ns, self.p, self.b_n)

In [ ]:
# # sersic_dens = SersicDensity(m=m_tot, Re=Re, ns=n_s)
# # sersic_dens = SersicDensity(m=m_tot*(1+z)**-0.67, 
#                             Re=R_e*(1+z)**-1.27, 
#                             ns=n_s*(1+z)**-0.95)

In [ ]:
# tot_mass,_ = quad(lambda r: 4*np.pi*r**2*sersic_dens(r), 0., np.inf)
# assert np.allclose(tot_mass, 5E10)

Try to recreate Figure 1 in G14

In [ ]:
from scipy.optimize import curve_fit

In [ ]:
derp_r = np.array([1., 10., 100.])
derp_dens = np.array([100., 1.8E-1, 3E-7])
ln_derp_r = np.log(derp_r)
ln_derp_dens = np.log(derp_dens)

def model(r, rho0, n_s, R_e):
    return rho0 * np.exp(-(r/R_e)**(1/n_s) + 1)

def ln_model(r, rho0, n_s, R_e):
    return np.log(model(r, rho0, n_s, R_e))

# p_fit,_ = curve_fit(ln_model, ln_derp_r, ln_derp_dens)
p_fit,_ = curve_fit(model, derp_r, derp_dens, p0=[100., 1.5, 1.], sigma=derp_dens/10.)

In [ ]:
pl.figure(figsize=(6,6))
# pl.loglog(r, model(r, 100., 1.5, 1.))
pl.loglog(r, model(r, *p_fit))
pl.loglog(derp_r, derp_dens, ls='none', marker='o', markersize=10)

pl.xlabel('$r$ [kpc]')

pl.xlim(1E-1, 1E2)
pl.ylim(1E-7, 1E2)

In [ ]:
inv_rho0,_ = quad(lambda r: 4*np.pi*r**2*model(r, 1., *p_fit[1:]), 0., np.inf)
model_p = [5E10/inv_rho0] + list(p_fit[1:])

tot_mass,_ = quad(lambda r: 4*np.pi*r**2*model(r, *model_p), 0., np.inf)
assert np.allclose(tot_mass, 5E10)

Use SCF / BFE to get potential and etc. for this density distribution

In [ ]:
(S,Serr),_ = bscf.compute_coeffs(lambda x,y,z: model(math.sqrt(x**2+y**2+z**2), *model_p), 
                                 nmax=16, lmax=0, M=m_tot, r_s=1.)

In [ ]:
pl.semilogy(np.abs(S[:,0,0]))

In [ ]:
r = np.logspace(-1, 2, 128) # kpc
xyz = np.zeros((r.size, 3))
xyz[:,0] = r
scf_dens = bscf.density(xyz, Snlm=S, Tnlm=np.zeros_like(S), M=m_tot, r_s=1.)

pl.loglog(r, model(r, *model_p))
pl.loglog(r, scf_dens)
pl.xlabel('$r$ [kpc]')
pl.ylabel('Density')

The potential used in G14 is:
- DM: spherical NFW halo with $M_h = 10^{12}~{\rm M}_\odot$, $r_s = 20~{\rm kpc}$
- Stars: a "spherical Sérsic profile" with $M = 5\times 10^{10}~{\rm M}_\odot$, $R_e = 4~{\rm kpc}$ and $n_s = 2.2$

In [ ]:
pot = gp.CCompositePotential()
pot['stars'] = bscf.SCFPotential(m=sersic_dens.m, r_s=sersic_dens.Re, 
                                 Snlm=S, Tnlm=np.zeros_like(S), units=galactic)

v_c = np.sqrt(G * 1E12*u.Msun / (20*u.kpc) * (np.log(2.)-0.5))
v_c = galactic.decompose(v_c)
print(v_c)
pot['halo'] = gp.SphericalNFWPotential(v_c=v_c, r_s=20*u.kpc, units=galactic) # M(<250kpc) = 10^12 Msun

pot.mass_enclosed([250.,0,0]*u.kpc) # mass in ~virial radius

In [ ]:
r = np.logspace(-1,2,256)
q = np.zeros((3,r.size))
q[0] = r

fig,axes = pl.subplots(1,2,figsize=(12,5))

axes[0].loglog(r, pot.density(q), marker=None, label='total', lw=2.)
axes[0].loglog(r, pot['stars'].density(q), marker=None, label='stars')
axes[0].loglog(r, pot['halo'].density(q), marker=None, label='dm')
axes[0].legend()

axes[0].set_xlabel('$r$ [kpc]')
axes[0].set_ylabel(r'Density [${\rm M}_\odot\,{\rm kpc}^{-3}$]')

axes[1].loglog(r, pot.mass_enclosed(q), marker=None, label='total', lw=2.)
axes[1].loglog(r, pot['stars'].mass_enclosed(q), marker=None, label='stars')
axes[1].loglog(r, pot['halo'].mass_enclosed(q), marker=None, label='dm')
axes[1].set_xlabel('$r$ [kpc]')
axes[1].set_ylabel(r'Mass enclosed [${\rm M}_\odot$]')

fig.tight_layout()

Circular velocity curve for the total potential

In [ ]:
def circ_vel(r, pot):
    q = np.zeros((3,r.size))*r.unit
    q[0] = r
    dPhi_dr = pot.gradient(q)[0]
    return np.sqrt(r*np.abs(dPhi_dr)).to(u.km/u.s)

In [ ]:
pl.plot(r, circ_vel(r*u.kpc, pot), marker=None)
pl.xlabel('$r$ [kpc]')
pl.ylabel(r'$v_{\rm c}(r)$ [km ${\rm s}^{-1}$]')

---

In [ ]:
N_gcs = 8000
t_evolve = 11.5*u.Gyr

## Sample masses from power law

In [ ]:
def sample_masses(M_min, M_max, size=1):
    M_min = M_min.to(u.Msun).value
    M_max = M_max.to(u.Msun).value
    
    class MassFunction(rv_continuous):
        def _pdf(self, x):
            return x**-2 / (1/self.a - 1/self.b)
        
    return MassFunction(a=M_min, b=M_max).rvs(size=size) * u.Msun

## Sample radii

In [ ]:
def sample_radii(pot, r_min=0.*u.kpc, r_max=np.inf*u.kpc, size=1):
    r_min = r_min.to(u.kpc).value
    r_max = r_max.to(u.kpc).value
    
    # Menc = lambda rr: quad(lambda r: 4*np.pi*r**2*pot._density(np.array([r,0,0])), 0, rr)[0]
    Mtot = pot.parameters['m'].value
    Menc = lambda rr: pot.mass_enclosed([rr,0.,0.]).value 
    
    def root_func(r, m):
        return (m - Menc(float(r))/Mtot)[0]
    
    if r_min == 0.:
        m_min = 0.
    else:
        m_min = Menc(r_min)/Mtot
        
    if r_max == np.inf:
        m_max = 1.
    else:
        m_max = Menc(r_max)/Mtot
    
    m = np.random.uniform(m_min, m_max, size=size)
    return np.array([root(root_func, 10., args=(m[i],)).x[0] for i in range(size)]) * u.kpc

In [ ]:
np.random.seed(seed)
gc_masses = sample_masses(M_min=1E4*u.Msun, M_max=1E7*u.Msun, size=N_gcs)
gc_r = sample_radii(pot['stars'], 0.1*u.kpc, 100*u.kpc, size=N_gcs)

Compare with Figure 2 in G14

In [ ]:
fig,axes = pl.subplots(1,2,figsize=(12,6))
axes[0].hist(gc_masses, bins=np.logspace(4,7.1,9));
axes[0].set_xscale('log')
axes[0].set_yscale('log')
axes[0].set_xlim(1E3, 3E7)
axes[0].set_ylim(5E-1, 1E4)
axes[0].set_xlabel(r"Mass [${\rm M}_\odot$]")
axes[0].set_ylabel(r"$N$")

bins = np.logspace(-1.,2.,16)
H,_ = np.histogram(gc_r, bins=bins)

V = 4/3*np.pi*(bins[1:]**3 - bins[:-1]**3)
bin_cen = (bins[1:]+bins[:-1])/2.
axes[1].plot(bin_cen, H/V, ls='--', marker=None)

axes[1].set_xscale('log')
axes[1].set_yscale('log')
axes[1].set_xlim(1E-1, 1E2)
axes[1].set_ylim(1E-7, 1E2)
axes[1].set_xlabel(r"$r$ [kpc]")
axes[1].set_ylabel('GC density [kpc$^{-3}$]')

fig.tight_layout()

## Compute disruption times

In [ ]:
# def P(r, pot):
#     return 41.4 * (r.to(u.kpc).value) / (circ_vel(r, pot).to(u.km/u.s).value)
    
# def t_tid(r, M, pot, α=2/3.):
#     """ Tidal disruption timescale """
#     return 10*u.Gyr * (M / (2E5*u.Msun))**α * P(r, pot)

# def t_iso(M):
#     """ Isolation disruption timescale (2-body evaporation)"""
#     return 17*u.Gyr * (M / (2E5*u.Msun))

# def t_df(r, M, pot, f_e=0.5):
#     return 0.45*u.Gyr * (r.to(u.kpc).value)**2 * (circ_vel(r, pot).to(u.km/u.s).value) * (M/(1E5*u.Msun))**-1 * f_e

# def F(y, t):
#     M,r2 = y
    
#     r = np.sqrt(r2)
#     min_t = np.min([t_tid(r*u.kpc, M*u.Msun, pot).to(u.Gyr).value, 
#                     t_iso(M*u.Msun).to(u.Gyr).value], axis=0)
    
#     M_dot = -M / min_t
#     r2_dot = -r2 / t_df(r*u.kpc, M*u.Msun, pot).to(u.Gyr).value
    
#     return [float(M_dot), float(r2_dot)]

In [ ]:
_fac = (1*u.kpc/u.Myr).to(u.km/u.s).value
def _circ_vel_kms(r, pot):
    r = float(r)
    dPhi_dr = pot._gradient(np.array([r,0.,0.]))[0]
    return np.sqrt(np.abs(r*dPhi_dr)) * _fac

def P(r, pot):
    return 41.4 * r / _circ_vel_kms(r, pot)
    
def t_tid(r, M, pot, α=2/3.):
    """ Tidal disruption timescale """
    return 10. * (M / 2E5)**α * P(r, pot)

def t_iso(M):
    """ Isolation disruption timescale (2-body evaporation)"""
    return 17. * (M / 2E5)

def t_df(r, M, pot, f_e=0.5):
    return 0.45 * r*r * _circ_vel_kms(r,pot) * (M/1E5)**-1 * f_e

def F(y, t):
    M,r2 = y
    
    r = np.sqrt(r2)
    min_t = np.min([t_tid(r, M, pot), 
                    t_iso(M)], axis=0)
    
    M_dot = -M/min_t
    r2_dot = -r2/t_df(r, M, pot)
    
    return [float(M_dot), float(r2_dot)]

In [ ]:
fname = "../output/gc-evolve.h5"
if not os.path.exists(fname) or True:
    t_grid = np.linspace(0., t_evolve.value, 4096)

    with h5py.File(fname, "w") as f:
        f.create_dataset('time', data=t_grid)
        f.create_group('mass')
        f.create_group('radius')

        f['time'].attrs['unit'] = 'Gyr'
        f['mass'].attrs['unit'] = 'solMass'
        f['radius'].attrs['unit'] = 'kpc'

        for i,_r,_M in zip(range(N_gcs), gc_r.to(u.kpc).value, gc_masses.to(u.Msun).value):
            if (i % 1000) == 0:
                print(i)

            M_r2 = odeint(F, [_M, _r**2], t=t_grid)
            f['mass'].create_dataset(str(i), data=M_r2[:,0])
            f['radius'].create_dataset(str(i), data=np.sqrt(M_r2[:,1]))

In [ ]:
t_disrupt = np.zeros(N_gcs)
final_r = np.zeros(N_gcs)
final_m = np.zeros(N_gcs)
did_disrupt = np.zeros(N_gcs).astype(bool)
with h5py.File("../output/gc-evolve.h5", "r") as f:
    for i in range(N_gcs):
        _mass = f['mass/{}'.format(i)][:]
        _radius = f['radius/{}'.format(i)][:]
        
        if np.any(np.isnan(_mass)) or np.any(_mass < 0.):
            t_disrupt[i] = f['time'][np.isfinite(_mass) & (_mass >= 0)][-1]
            final_r[i] = _radius[np.isfinite(_mass) & (_mass >= 0)][-1]
            final_m[i] = 0.
            did_disrupt[i] = True
            
        else:
            t_disrupt[i] = np.nan
            final_r[i] = _radius[-1]
            final_m[i] = _mass[-1]
            did_disrupt[i] = False
            
    t_disrupt = t_disrupt*u.Unit(f['time'].attrs['unit'])
    final_r = final_r*u.Unit(f['radius'].attrs['unit'])

In [ ]:
print("{} did not disrupt".format(N_gcs-did_disrupt.sum()))
print("{:.0%}".format(did_disrupt.sum() / N_gcs))

In [ ]:
pl.figure(figsize=(6,5))
hist((t_disrupt - t_evolve)[did_disrupt], bins='scott')
pl.yscale('log')
pl.xlabel('Disruption time [Gyr]')
pl.ylabel('$N$')
pl.tight_layout()
# pl.savefig("/Users/adrian/projects/how-many-streams/plots/t_disrupt.pdf")

TODO: now i have disruption times, masses, radii, and mass-loss rates I can run simulations with some eccentricity distribution

---

In [ ]:
fig,axes = pl.subplots(1,2,figsize=(12,6))

axes[0].scatter(gc_masses, final_m, c=np.log10(gc_r.value), cmap='magma_r', alpha=0.75, s=4)
axes[0].set_xscale('log')
axes[0].set_yscale('log')
axes[0].set_xlim(1E2,1E7)
axes[0].set_ylim(1E2,1E7)
axes[0].set_xlabel('$M_i$ [kpc]')
axes[0].set_ylabel('$M_f$ [kpc]')

axes[1].scatter(gc_r, final_r, c=np.log10(gc_masses.value), cmap='magma_r', alpha=0.75, s=4)
axes[1].set_xlim(0,50)
axes[1].set_ylim(0,50)
axes[1].set_xlabel('$r_i$ [kpc]')
axes[1].set_ylabel('$r_f$ [kpc]')

In [ ]:
from astropy.io import fits
import astropy.coordinates as coord

In [ ]:
harris_tbl = fits.getdata("../data/harris-gc-catalog.fits", 1)

In [ ]:
harris_c = coord.SkyCoord(ra=harris_tbl['RA']*u.degree, dec=harris_tbl['DEC']*u.degree,
                          distance=harris_tbl['HELIO_DISTANCE']*u.kpc)
harris_gc_dist = harris_c.transform_to(coord.Galactocentric).represent_as(coord.PhysicsSphericalRepresentation).r

In [ ]:
fig,axes = pl.subplots(1,2,figsize=(12,6))

axes[0].hist(gc_masses, bins=np.logspace(4,7.1,9));
axes[0].hist(final_m[~did_disrupt], bins=np.logspace(3,7.1,12));

axes[0].set_xscale('log')
axes[0].set_yscale('log')
axes[0].set_xlim(1E3, 3E7)
axes[0].set_ylim(5E-1, 1E4)
axes[0].set_xlabel(r"Mass [${\rm M}_\odot$]")
axes[0].set_ylabel(r"$N$")

bins = np.logspace(-1.,2.,16)
H,_ = np.histogram(gc_r, bins=bins)
data_H,_ = np.histogram(harris_gc_dist, bins=bins)

V = 4/3*np.pi*(bins[1:]**3 - bins[:-1]**3)
bin_cen = (bins[1:]+bins[:-1])/2.
axes[1].plot(bin_cen, H/V, ls='--', marker=None)
axes[1].errorbar(bin_cen, data_H/V, np.sqrt(data_H)/V, 
                 color='k', marker='o', ecolor='#666666', linestyle='none')

H_f,_ = np.histogram(final_r[~did_disrupt], bins=bins)
axes[1].plot(bin_cen, H_f/V, ls='--', marker=None)

axes[1].set_xscale('log')
axes[1].set_yscale('log')
axes[1].set_xlim(1E-1, 1E2)
axes[1].set_ylim(1E-7, 1E2)
axes[1].set_xlabel(r"$r$ [kpc]")
axes[1].set_ylabel('GC density [kpc$^{-3}$]')

fig.tight_layout()

fig.savefig("/Users/adrian/projects/how-many-streams/plots/GC-mass-radius.pdf")

---

# Ignore below here

---

### For a given mass-loss history and radius, estimate the length of the stream

$$
\begin{align}
l &= 2\Delta \theta\\
\Delta \theta_i &= \Delta\Omega_i \, t\\
\Delta\Omega_i &\approx \Omega_i \, \frac{\delta E}{E}\\
\delta E &= r_{\rm tid} \, \frac{d\Phi}{dr}\\
r_{\rm tid} &\approx \left(\frac{M_{\rm GC}}{3M(<r)}\right)^{1/3} \, r
\end{align}
$$
$\epsilon$ is the energy scale, $r_{\rm tid}$ is the tidal radius, $M(<r)$ is the enclosed mass of the Galaxy at radius $r$.

In [ ]:
def delta_Omega(r, M, pot):
    vc = circ_vel(r, pot)
    Omega = vc / r
    
    q = np.zeros(3)*r.unit
    q[0] = r
    
    # p = np.zeros(3)*vc.unit
    # p[0] = vc
    
    # r_tid = (M / (3*pot.mass_enclosed(q)))**(1/3.) * r
    # dE = r_tid * pot.gradient(q)[0]
    # E = pot.total_energy(q, p)
    dE_E = (M / (3*pot.mass_enclosed(q)))**(1/3.)
    
    return (dE_E*Omega)[0]

def get_r_tid(r, M, pot):
    r = np.atleast_1d(r)
    M = np.atleast_1d(M)
    
    q = np.zeros((3,r.size))*r.unit
    q[0] = r
    return (M / (3*pot.mass_enclosed(q)))**(1/3.) * r

In [ ]:
l = (delta_Omega(gc_r[0], gc_mass[0], pot) * (t_evolve-t_disrupt[0]))\
    .to(u.degree, equivalencies=u.dimensionless_angles())
l

TODO: As a really crude initial estimate (assuming spherical bg), if I know how much mass is lost at each time-step I can then allow it to linearly evolve in angle with a Gaussian cylinder radius set by the tidal radius. From this I can get the surface density. For example, the mass-loss info would give me a "strand" of stars, then I would convolve with a Gaussian to get a "stream"

In [ ]:
def kernel(r_tid, w):
    K = np.eye(3)
    K[0,0] = w.to(u.kpc).value
    K[1,1] = w.to(u.kpc).value
    K[2,2] = r_tid.to(u.kpc).value
    return K

def get_grid(r_tid, z_theta, dd=None):
    r_tid = r_tid.to(u.kpc).value
    z_theta = z_theta.to(u.kpc).value
    
    max_r_tid = np.nanmax(r_tid)
    max_z_theta = np.nanmax(z_theta)
    
    if dd is None:
        dd = max_r_tid.max()/4.
    
    print(np.arange(-3*max_r_tid, 3*max_r_tid+dd, dd).size, 
          np.arange(0., max_z_theta+max_r_tid + dd, dd).size)
    return
    dens_grid = np.meshgrid((np.arange(-5*r_tid, 5*r_tid+dd, dd),
                             np.arange(-5*r_tid, 5*r_tid+dd, dd),
                             np.arange(0., z_theta.max()+r_tid + dd, dd)))
    return dens_grid

In [ ]:
with h5py.File("../output/gc-evolve.h5", "r") as f:
    time = f['time'][:] * u.Unit(f['time'].attrs['unit'])
    t_evolve = time.max()
#     t_evolve = 2.5*u.Gyr

    for i in range(N_gcs): #[2455:2456]:
        M_i = f['mass/{}'.format(i)][:] * u.Unit(f['mass'].attrs['unit'])
        r_i = f['radius/{}'.format(i)][:] * u.Unit(f['radius'].attrs['unit'])
        
        dM = M_i[1:] - M_i[:-1]
        mean_M = (M_i[:-1] + M_i[1:])/2.
        mean_r = (r_i[:-1] + r_i[1:])/2.
        
        theta = np.nan*np.zeros(len(dM))*u.degree
        for j in range(len(dM)):
            dt = t_evolve-time[j]
            if dt < 0.:
                break
            
            theta[j] = (delta_Omega(mean_r[j], mean_M[j], pot) * dt)\
                       .to(u.degree, equivalencies=u.dimensionless_angles())
            
            if M_i[j] < 0.:
                break
            
        # turn angular coordinate into cartesian
        z_theta = (mean_r * theta).to(u.kpc, equivalencies=u.dimensionless_angles())

#         r_tid = get_r_tid(mean_r, mean_M, pot)
#         get_grid(r_tid, z_theta)
        # need to estimate mass density from this -- TODO: need width as a function of theta!
        
        #idx = np.isfinite(Omega)
        #H,edges = np.histogram(Omega[idx], bins=np.linspace(0.,180,180), weights=-dM[idx])

        break

In [ ]:
idx = np.isfinite(theta)
H,edges = np.histogram(theta[idx], bins=np.linspace(0.,420,64), weights=-dM[idx])
# H,edges = np.histogram(z_theta[idx], bins=np.linspace(0.,15,64), weights=-dM[idx])

pl.plot(edges[1:], H)

In [ ]:
print("hi")